<a href="https://colab.research.google.com/github/Monstermel/NBA-Web-Scrapper/blob/main/Web_scraper_nba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalacion de Selenium

In [ ]:
!apt-get update
!pip3 install selenium

#Importacion de librerias

In [3]:
import os
import csv
import json
import time
import requests

from itertools import islice
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

#Url, Payload y Headers

In [4]:
playergamelogs = 'https://stats.nba.com/stats/playergamelogs'

In [5]:
payload =  {
    'DateFrom': '',
    'DateTo': '',
    'GameSegment': '',
    'LastNGames': '0',
    'LeagueID': '00',
    'Location': '',
    'MeasureType': 'Base',
    'Month': '0',
    'OppTeamID': '0',
    'OpponentTeamID': '0',
    'Outcome': '',
    'PORound': '0',
    'PaceAdjust': 'N',
    'PerMode': 'Totals',
    'Period': '0',
    'PlayerID': '',
    'PlusMinus': 'N',
    'Rank': 'N',
    'Season': '',
    'SeasonSegment': '',
    'SeasonType': 'Regular Season',
    'ShotClockRange': '',
    'TeamID': '',
    'VsConference': '',
    'VsDivision': ''
}

In [82]:
headers = {
    'Accept':'*/*',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'en-US,en;q=0.9,es-MX;q=0.8,es;q=0.7',
    'Connection':'keep-alive',
    'Dnt':'1',
    'Host':'stats.nba.com',
    'Origin':'https://www.nba.com',
    'Referer':'https://www.nba.com/',
    'Sec-Ch-Ua':'"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'Sec-Ch-Ua-Mobile':'?0',
    'Sec-Ch-Ua-Platform':'"Windows"',
    'Sec-Fetch-Dest':'empty',
    'Sec-Fetch-Mode':'cors',
    'Sec-Fetch-Site':'same-site',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

#Script

In [7]:
def get_player_ids():
    url = 'https://www.nba.com/players'

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    element = driver.find_element(By.XPATH, '//select[@title="Page Number Selection Drown Down List"]')
    driver.execute_script('arguments[0].selectedIndex = 0', element)
    driver.execute_script('arguments[0].dispatchEvent(new Event("change", {"bubbles": true}));', element)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    links = soup.select('.RosterRow_playerLink__qw1vG')

    ids = []
    for link in links:
        ids.append(link.get('href').split('/')[2])

    driver.quit()
    return ids

In [21]:
def get_seasons_by_player(id: int):
    url = f'https://www.nba.com/stats/player/{id}/boxscores-traditional'

    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        selector = soup.select_one('.DropDown_select__4pIg9')
        seasons = []
        try:
            for option in selector.select("option"):
                seasons.append(option.text)
            break
        except:
            print("[Error processing seasons]...", end='')

    return seasons

In [1]:
def get_data(start=None, end=None, wait_player=3, wait_season=1):
    if not os.path.exists('data/'):
        os.makedirs('data/')

    print("Getting players ids.........................................", end='')
    player_lst = get_player_ids()
    print("done")

    total_players = len(player_lst)
    for idx, player in islice(enumerate(player_lst), start, end):
        print(f'Processing player {idx+1}/{total_players}.....................................', end='')
        payload_aux = payload
        payload_aux['PlayerID'] = str(player)

        seasons = get_seasons_by_player(player)
        time.sleep(wait_season)

        for season in seasons:
            payload_aux['Season'] = str(season)
            response = requests.get(playergamelogs, headers = headers, params = payload_aux, timeout = 30)
            time.sleep(wait_season)

            if response.status_code == 200:
                data = response.json()
            else:
                print('[Error: ' + str(response.status_code) + ']...', end='')
                continue

            if not os.path.exists(f'data/{player}/'):
                os.makedirs(f'data/{player}/')

            with open(f'data/{player}/{player}_{season}.csv', 'w', newline='') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(data['resultSets'][0]['headers'])

                for row in data['resultSets'][0]['rowSet']:
                    writer.writerow(row)

        time.sleep(wait_player)
        print('done')

In [ ]:
get_data()

In [ ]:
!find data/ -type d | tail -n +2 | wc -l

In [ ]:
!cp -r data /content/drive/MyDrive/data

#Juntando los datos

In [ ]:
%%shell
find data/ -type f | sort -V > dirs.txt

head -1 $(head -1 dirs.txt) > "player_stats.csv"

for file in $(cat dirs.txt); do
    tail -n +2 "$file" >> "player_stats.csv";
done

In [17]:
!cp player_stats.csv drive/MyDrive/data/